# MERGE Algorithm Walkthrough
This is a reimplementation of the MERGE algorithm. My hope is that by proceeding through the algorithm in a linear fashion, the order of operations will be clearer and will facilitate a refactoring into more generalized code.

## Initialization

When the model is initialized, it needs three things: a pre-processed corpus, a gapsize, and the number of iterations. We'll use the corpus included in the original implementation, which is "[...] a combination of the Santa Barbara Corpus of Spoken American English and the spoken component of the ICE Canada corpus"

> Du Bois, John W., Wallace L. Chafe, Charles Meyers, Sandra A. Thompson, Nii Martey, and Robert Englebretson (2005). Santa Barbara corpus of spoken American English. Philadelphia: Linguistic Data Consortium.

> Newman, John and Georgie Columbus (2010). The International Corpus of English – Canada. Edmonton, Alberta: University of Alberta.


Note that these are dialog corpuses, so each line is often referred to as a `turn`. They've also been pre-processed with lowercasing, and punctuation replaced with alphanumeric substitutions (`_` --> `undrscr`).


In [1]:
from pathlib import Path
from itertools import chain
import re

# these are from the original implementation, but we get the same line count without them
# so they're here for reference
delimiters = set([" ", ".", ",", "?", ";", ":", "!", "\r", "\n"])

corpus = []
for txt_file in Path("Combined_corpus/").glob("*.TXT"):
    for line in txt_file.read_text().split("\n"):
        if line:
            corpus.append(line.split(" "))

corpus_size = len(list(chain.from_iterable(corpus)))
print(
    f"Lines (turns): {len(corpus):,}",
)
print(f"Tokens (corpus_size in code): {corpus_size:,}")


Lines (turns): 62,676
Tokens (corpus_size in code): 844,510


In [2]:
# Then store the tokens in the special data structure

from collections import Counter, defaultdict
from itertools import product
from typing import (
    Dict,
    Generator,
    List,
    NamedTuple,
    Set,
    Tuple,
    DefaultDict,
    Counter as CounterType,
    NewType,
)


In [3]:
# We create some simple data structures


class Word(NamedTuple):
    wordstr: str
    position: int


# Lexeme can be used to store n-grams as the algorithm works
# plus we need to hash it so we use an expanding tuple


class Lexeme(NamedTuple):
    word: Tuple[Word, ...]
    token_index: int


# These are not in original code but we refer to them alot, so they will
# be helpful as types.

LineIndex = NewType("LineIndex", int)
TokenIndex = NewType("TokenIndex", int)

# In the original code there is a Lexemes object that stores all this data
# in an effort to flatten and linearize the code, I wont define or use those
# objects here

# Lexemes
_lexemes_to_locations: DefaultDict[
    Lexeme, Set[Tuple[LineIndex, TokenIndex]]
] = defaultdict(set)
_locations_to_lexemes: DefaultDict[LineIndex, Dict[TokenIndex, Lexeme]] = defaultdict(
    dict
)
_locations_to_locations: Dict[
    Tuple[LineIndex, TokenIndex], Tuple[LineIndex, TokenIndex]
] = {}
turn_lengths: CounterType[int] = Counter()

# NOTE: This is a Counter in original code, but doesn't use counter methods.
# and is typed as a regular dict here
_lexemes_to_freqs: Dict[Lexeme, int] = {}


In [4]:
for line_ix, line in enumerate(corpus):
    for word_ix, word in enumerate(line):
        lexeme = Lexeme(word=(Word(word, 0),), token_index=0)
        loc: Tuple[LineIndex, TokenIndex] = (line_ix, word_ix)
        _lexemes_to_locations[lexeme].add(loc)
        _locations_to_lexemes[line_ix][word_ix] = lexeme
        _locations_to_locations[
            loc
        ] = loc  # from original code, not sure what this does
        turn_lengths[line_ix] += 1  # TODO: Check out where this is used

# Lexemes.count_frequencies()
_lexemes_to_freqs = {k: len(v) for k, v in _lexemes_to_locations.items()}

# We could double check this as a counter now
Counter(_lexemes_to_freqs).most_common(5)


[(Lexeme(word=(Word(wordstr='the', position=0),), token_index=0), 35262),
 (Lexeme(word=(Word(wordstr='and', position=0),), token_index=0), 26174),
 (Lexeme(word=(Word(wordstr='i', position=0),), token_index=0), 21216),
 (Lexeme(word=(Word(wordstr='to', position=0),), token_index=0), 19509),
 (Lexeme(word=(Word(wordstr='you', position=0),), token_index=0), 18314)]

# Bigram Data
Bigram data is stored in another object called `Bigrams`. There is a lot of logic in here we are going to separate out. 

We also need to declare a `gapsize` at this point, which allows for MWEs with discontinuity (e.g. `in _ of`). 

In [5]:
Gapsize = NewType("Gapsize", int)

gapsize = Gapsize(1)


In [6]:
# Data structures
from dataclasses import dataclass, field


class Bigram(NamedTuple):
    el1: Lexeme
    el2: Lexeme
    gapsize: int


def defaultdict_set_factory():
    return defaultdict(set)


@dataclass
class BigramData:
    bigrams_to_freqs: CounterType[Bigram] = field(default_factory=Counter)
    bigrams_to_locations: Dict[Bigram, Set[Tuple[LineIndex, TokenIndex]]] = field(
        default_factory=lambda: defaultdict(set)
    )
    left_lex_to_bigrams: Dict[Bigram, Set[Tuple[Lexeme, Gapsize]]] = field(
        default_factory=lambda: defaultdict(set)
    )
    right_lex_to_bigrams: Dict[Bigram, Set[Tuple[Lexeme, Gapsize]]] = field(
        default_factory=lambda: defaultdict(set)
    )
    type_count: int = 0


# Maybe put these in a container

initial_bigrams = BigramData()


In [7]:
line_gap_product = product(*(turn_lengths.items(), range(gapsize + 1)))
# This could just be a nested for loop, but we find the cartesian product
# we essentially want to loop over each line and each gapsize
for (turnindex, turnlength), curr_gapsize in line_gap_product:
    # rightmost_leftedge in code
    last = turnlength - curr_gapsize - 1
    # last token that can be part of a bigram. Think if gapsize = 0,
    # the the second to last token will be the first element of
    # the final bigram
    # NOTE: we arent subtracting (-2) beacuse length is already
    # iterables + 1
    for ix in range(last):
        _left = _locations_to_lexemes[turnindex][ix]
        _right = _locations_to_lexemes[turnindex][ix + curr_gapsize + 1]
        _location = _locations_to_locations.get((turnindex, ix), (turnindex, ix))
        bgr = Bigram(el1=_left, el2=_right, gapsize=curr_gapsize)
        if bgr not in initial_bigrams.bigrams_to_freqs:
            initial_bigrams.type_count += 1
            initial_bigrams.left_lex_to_bigrams[(_left, curr_gapsize)].add(bgr)
            initial_bigrams.right_lex_to_bigrams[(_right, curr_gapsize)].add(bgr)
        initial_bigrams.bigrams_to_freqs[bgr] += 1
        initial_bigrams.bigrams_to_locations[bgr].add(_location)
        

## Candidate Table
The canddiate table is a pandas dataframe that is used to store frequency information. In the original implementation it's something like a paged data frame with multiple `tables`. I don't see the reason for this so we'll simplify the implementation into a single dataframe.

In [8]:
import pandas as pd

data = []
for index, (bgr, freq) in enumerate(initial_bigrams.bigrams_to_freqs.items()):
    # Look up the frequency of each element of the bigrams
    el1_freq = _lexemes_to_freqs[bgr.el1]
    el2_freq = _lexemes_to_freqs[bgr.el2]
    row = {"bgr": bgr, "bgr_freq": freq, "el1_freq": el1_freq, "el2_freq": el2_freq}
    data.append(row)

table = pd.DataFrame(data).set_index("bgr")
table_og = table.copy()


In [9]:
table.head()


,bgr_freq,el1_freq,el2_freq
bgr,,,
"(((Word(wordstr='so', position=0),), 0), ((Word(wordstr='you', position=0),), 0), 0)",351,6702,18314
"(((Word(wordstr='you', position=0),), 0), ((Word(wordstr='donundrscrt', position=0),), 0), 0)",513,18314,3476
"(((Word(wordstr='donundrscrt', position=0),), 0), ((Word(wordstr='need', position=0),), 0), 0)",50,3476,577
"(((Word(wordstr='need', position=0),), 0), ((Word(wordstr='to', position=0),), 0), 0)",205,577,19509
"(((Word(wordstr='to', position=0),), 0), ((Word(wordstr='go', position=0),), 0), 0)",583,19509,2231


## Iterating the algorithm

Now the fun begins. The first thing we do is calculate the 'winner' from the bigram tables by calculating the log-likelihood for each bigram. The bigram with the highest log-likelihood is the winner, and will then be merged.

In [10]:
# TODO: Validate this specific implementation of log-likelihood

import numpy as np

obsA = table["bgr_freq"]
obsB = table["el1_freq"] - table["bgr_freq"]
obsC = table["el2_freq"] - table["bgr_freq"]
obsD = corpus_size - (obsA + obsB + obsC)

expA = table["el2_freq"] / corpus_size * table["el1_freq"]
expB = (corpus_size - table["el2_freq"]) / corpus_size * table["el1_freq"]
expC = table["el2_freq"] / corpus_size * (corpus_size - table["el1_freq"])
expD = (
    (corpus_size - table["el2_freq"]) / corpus_size * (corpus_size - table["el1_freq"])
)

llA = obsA * np.log1p(obsA / expA)

real_vals = np.where(obsB != 0)[0]
llB = np.zeros(len(table))
llB[real_vals] = obsB[real_vals] * np.log(obsB[real_vals] / expB[real_vals])

real_vals = np.where(obsC != 0)[0]
llC = np.zeros(len(table))
llC[real_vals] = obsC[real_vals] * np.log(obsC[real_vals] / expC[real_vals])

llD = obsD * np.log(obsD / expD)

log_likelihood = 2 * (llA + llB + llC + llD)
negs = np.where(llA < 0)[0]
log_likelihood[negs] = log_likelihood[negs] * -1


In [11]:
# NOTE: This was checked against the implementation + was correct.

log_likelihood.sort_values().tail(5)


bgr
(((Word(wordstr='i', position=0),), 0), ((Word(wordstr='donundrscrt', position=0),), 0), 0)     9586.453385
(((Word(wordstr='i', position=0),), 0), ((Word(wordstr='mean', position=0),), 0), 0)            9674.049457
(((Word(wordstr='i', position=0),), 0), ((Word(wordstr='think', position=0),), 0), 0)           9868.626038
(((Word(wordstr='mhh', position=0),), 0), ((Word(wordstr='hmm', position=0),), 0), 0)          10541.817803
(((Word(wordstr='you', position=0),), 0), ((Word(wordstr='know', position=0),), 0), 0)         28104.733018
dtype: float64

In [12]:
winner_info: Bigram = log_likelihood.idxmax()
winner_info


Bigram(el1=Lexeme(word=(Word(wordstr='you', position=0),), token_index=0), el2=Lexeme(word=(Word(wordstr='know', position=0),), token_index=0), gapsize=0)

## Merging
We now create the merged token, which will then be substituted into the original corpus where it occured. 

In [13]:
def merge_bigram_to_lexeme(bigram: Bigram) -> Lexeme:
    el1_words = list(bigram.el1.word)
    el2_words = [
        Word(wordstr=word, position=(pos + bigram.gapsize + 1))
        for (word, pos) in bigram.el2.word
    ]
    all_words = sorted(el1_words + el2_words, key=lambda word: word[1])
    new_lexeme = Lexeme(word=tuple(all_words), token_index=0)
    return new_lexeme


merge_token = merge_bigram_to_lexeme(winner_info)
merge_tracker: List[Lexeme] = []  # This is a Dict[iter, Lexeme] in the source
merge_tracker.append(merge_token)
merge_token


Lexeme(word=(Word(wordstr='you', position=0), Word(wordstr='know', position=1)), token_index=0)

## Bigram Updater

This is the most complex part of the algorithm. The steps of the bigram updater are:

- `Lexemes.set_merge_token` - creates `satellite_lexemes` a dict of position to new `Lexeme` with `token_index`=`position`



In [14]:
satellite_lexemes = {0: merge_token}
for wordstr, position in merge_token.word:
    if position > 0:
        satellite_lex = Lexeme(word=merge_token.word, token_index=position)
        satellite_lexemes[position] = satellite_lex

satellite_lexemes


{0: Lexeme(word=(Word(wordstr='you', position=0), Word(wordstr='know', position=1)), token_index=0),
 1: Lexeme(word=(Word(wordstr='you', position=0), Word(wordstr='know', position=1)), token_index=1)}

In [15]:
# iterate through winner locations for this next part

ContextPosition = NewType("ContextPosition", int)
SatellitePosition = NewType("SatellitePosition", int)


class ContextPos(NamedTuple):
    context_pos: ContextPosition
    satellite_pos: SatellitePosition
    gapsize: Gapsize
    merge_token_leftanchor: TokenIndex


merged_satellite_positions: Dict[Tuple[LineIndex, SatellitePosition], TokenIndex] = {}

winner_locations = initial_bigrams.bigrams_to_locations[winner_info]
merge_token_count = 0

new_bigrams = BigramData()
conflicting_bigrams = BigramData()

for (line_ix, word_ix) in winner_locations:
    # Check if it's among the existing conflicting bigrams
    # the first iteration we wont have this, so we'll come back to it
    # TODO: `bigram_is_among_existing_conflicting_bigrams`
    # Which is just a lookup of the winner in the `conflicting bigrams` table
    if (line_ix, word_ix) in conflicting_bigrams.bigrams_to_locations[winner_info]:
        # BigramUpdater.bigram_is_among_existing_conflicting_bigrams
        continue

    merge_token_count += 1

    satellite_positions = [word_ix + word.position for word in merge_token.word]

    # context_positions = BigramUpdater.context_pos_manager.generate_positions_around_satellites
    context_positions: List[ContextPos] = []
    curr_turn_length = turn_lengths[line_ix]
    for satellite_position in satellite_positions:
        for curr_gapsize in range(gapsize + 1):
            left_context_position = satellite_position - curr_gapsize - 1
            if left_context_position >= 0:
                curr_contextpos = ContextPos(
                    left_context_position, satellite_position, curr_gapsize, word_ix
                )
                context_positions.append(curr_contextpos)
            right_context_position = satellite_position + curr_gapsize + 1
            if right_context_position < curr_turn_length:
                curr_contextpos = ContextPos(
                    right_context_position, satellite_position, curr_gapsize, word_ix
                )
                context_positions.append(curr_contextpos)

    # BigramUpdater.vicinity_lex_manager.create_bigrams_with_lexemes_surrounding_satellites(
    # merged_satellite_positions is an empty dict on iter = 0 but referenced here
    for context_position_info in context_positions:

        # Context_position_info gets unpacked in original code if you're looking for those variables
        satellite_position = context_position_info.satellite_pos
        context_pos = context_position_info.context_pos
        if context_pos in satellite_positions:
            # TODO: Investigate what this does, without it a bunch
            # of token counts are added
            continue
        # (self.premerge_lexeme, self.premerge_leftanchor) = self.all_lexemes.get_lexeme(self.turn_number, self.satellite_position)
        # these _private variables are a re-implementation of Lexemes.get_lexeme
        _ref_lexeme = _locations_to_lexemes[line_ix][satellite_position]
        _ref_pos_in_turn = satellite_position - _ref_lexeme.token_index
        _leftanchor_lexeme = _locations_to_lexemes[line_ix][_ref_pos_in_turn]

        premerge_lexeme = _leftanchor_lexeme
        premerge_leftanchor = _ref_pos_in_turn
        context_loc = (line_ix, context_pos)

        if context_loc in merged_satellite_positions:
            context_lexeme = merge_token
            context_ix = merged_satellite_positions[context_loc]
        else:
            # TODO: another re-implementation of Lexemes.get_lexeme(turn, word)
            _context_ref_lexeme = _locations_to_lexemes[line_ix][context_pos]
            _context_ref_pos_in_turn = context_pos - _context_ref_lexeme.token_index
            _context_leftanchor_lexeme = _locations_to_lexemes[line_ix][
                _context_ref_pos_in_turn
            ]
            context_lexeme = _context_leftanchor_lexeme
            context_ix = _context_ref_pos_in_turn

        # BigramUpdater.create_new_bigram()
        left_context = context_ix < word_ix
        if left_context:
            # all_lexemes.get_extant_loc_object(turn_number, context_ix )
            location_tuple = _locations_to_locations.get(
                (line_ix, context_ix), (line_ix, context_ix)
            )
            gap_between_anchors = word_ix - context_ix - 1
            _left, _right = context_lexeme, merge_token
        else:  # right context
            location_tuple = _locations_to_locations.get(
                (line_ix, word_ix), (line_ix, word_ix)
            )
            gap_between_anchors = (
                context_ix - word_ix - 1
            )  # order reversed from left_context
            _left, _right = merge_token, context_lexeme
        # reimplemtation of Bigrams.save_bigram_data
        bgr = Bigram(el1=_left, el2=_right, gapsize=gap_between_anchors)
        if bgr not in new_bigrams.bigrams_to_freqs:
            new_bigrams.type_count += 1
            new_bigrams.left_lex_to_bigrams[(_left, gap_between_anchors)].add(bgr)
            new_bigrams.right_lex_to_bigrams[(_right, gap_between_anchors)].add(bgr)
        new_bigrams.bigrams_to_freqs[bgr] += 1
        new_bigrams.bigrams_to_locations[bgr].add(location_tuple)

        # BigramUpdater.create_conflicting_bigram()
        left_context = context_ix < premerge_leftanchor
        if left_context:
            # all_lexemes.get_extant_loc_object(turn_number, context_ix )
            location_tuple = _locations_to_locations.get(
                (line_ix, context_ix), (line_ix, context_ix)
            )
            gap_between_anchors = premerge_leftanchor - context_ix - 1
            _left, _right = context_lexeme, premerge_lexeme
        else:  # right context
            location_tuple = _locations_to_locations.get(
                (line_ix, premerge_leftanchor), (line_ix, premerge_leftanchor)
            )
            gap_between_anchors = (
                context_ix - premerge_leftanchor - 1
            )  # order reversed from left_context
            _left, _right = premerge_lexeme, context_lexeme
        # reimplemtation of Bigrams.save_bigram_data
        bgr = Bigram(el1=_left, el2=_right, gapsize=gap_between_anchors)
        if bgr not in conflicting_bigrams.bigrams_to_freqs:
            conflicting_bigrams.type_count += 1
            conflicting_bigrams.left_lex_to_bigrams[(_left, gap_between_anchors)].add(
                bgr
            )
            conflicting_bigrams.right_lex_to_bigrams[(_right, gap_between_anchors)].add(
                bgr
            )
        conflicting_bigrams.bigrams_to_freqs[bgr] += 1
        conflicting_bigrams.bigrams_to_locations[bgr].add(location_tuple)

    for satellite_position in satellite_positions:
        merged_satellite_positions[(line_ix, satellite_position)] = word_ix


In [16]:
len(conflicting_bigrams.bigrams_to_freqs)


5207

In [17]:
type_count_check = (new_bigrams.type_count, conflicting_bigrams.type_count)
original_impl = (3531, 5207)
print(
    f"new_bigrams_diff: {original_impl[0]-type_count_check[0]}, conflicting_bigrams_diff: {original_impl[1]-type_count_check[1]}"
)
# after iter 1
# AssertionError: new_bigrams_diff: 2, conflicting_bigrams_diff: 6
# TODO: Come back to this


new_bigrams_diff: 0, conflicting_bigrams_diff: 0


In [18]:
conflicting_bigrams.bigrams_to_locations[winner_info]


set()

In [19]:
winner_info


Bigram(el1=Lexeme(word=(Word(wordstr='you', position=0),), token_index=0), el2=Lexeme(word=(Word(wordstr='know', position=0),), token_index=0), gapsize=0)

In [20]:
conflicting_bigrams.bigrams_to_freqs.most_common(5)


[(Bigram(el1=Lexeme(word=(Word(wordstr='you', position=0),), token_index=0), el2=Lexeme(word=(Word(wordstr='i', position=0),), token_index=0), gapsize=1),
  305),
 (Bigram(el1=Lexeme(word=(Word(wordstr='know', position=0),), token_index=0), el2=Lexeme(word=(Word(wordstr='i', position=0),), token_index=0), gapsize=0),
  305),
 (Bigram(el1=Lexeme(word=(Word(wordstr='you', position=0),), token_index=0), el2=Lexeme(word=(Word(wordstr='and', position=0),), token_index=0), gapsize=1),
  238),
 (Bigram(el1=Lexeme(word=(Word(wordstr='know', position=0),), token_index=0), el2=Lexeme(word=(Word(wordstr='and', position=0),), token_index=0), gapsize=0),
  238),
 (Bigram(el1=Lexeme(word=(Word(wordstr='you', position=0),), token_index=0), el2=Lexeme(word=(Word(wordstr='what', position=0),), token_index=0), gapsize=1),
  235)]

In [21]:
new_bigrams.bigrams_to_freqs.most_common(5)


[(Bigram(el1=Lexeme(word=(Word(wordstr='you', position=0), Word(wordstr='know', position=1)), token_index=0), el2=Lexeme(word=(Word(wordstr='i', position=0),), token_index=0), gapsize=1),
  610),
 (Bigram(el1=Lexeme(word=(Word(wordstr='you', position=0), Word(wordstr='know', position=1)), token_index=0), el2=Lexeme(word=(Word(wordstr='and', position=0),), token_index=0), gapsize=1),
  476),
 (Bigram(el1=Lexeme(word=(Word(wordstr='you', position=0), Word(wordstr='know', position=1)), token_index=0), el2=Lexeme(word=(Word(wordstr='what', position=0),), token_index=0), gapsize=1),
  470),
 (Bigram(el1=Lexeme(word=(Word(wordstr='and', position=0),), token_index=0), el2=Lexeme(word=(Word(wordstr='you', position=0), Word(wordstr='know', position=1)), token_index=0), gapsize=0),
  440),
 (Bigram(el1=Lexeme(word=(Word(wordstr='you', position=0), Word(wordstr='know', position=1)), token_index=0), el2=Lexeme(word=(Word(wordstr='the', position=0),), token_index=0), gapsize=1),
  358)]

In [22]:
merge_token_count


4652

In [23]:
satellite_lexemes


{0: Lexeme(word=(Word(wordstr='you', position=0), Word(wordstr='know', position=1)), token_index=0),
 1: Lexeme(word=(Word(wordstr='you', position=0), Word(wordstr='know', position=1)), token_index=1)}

In [24]:
# BigramUpdater.update_all_lexemes_with_merge_tokens()
# (after main_control_loop)
_lexemes_to_freqs[merge_token] = merge_token_count
for (line_ix, satellite_pos), word_ix in merged_satellite_positions.items():
    # all_lexemes.add_merge_token(turn, satellite_pos, merge_token_leftanchor)
    loc = _locations_to_locations[(line_ix, satellite_pos)]
    satellite_lexeme = satellite_lexemes[satellite_pos - word_ix]
    _lexemes_to_locations[satellite_lexeme].add(loc)
    _locations_to_lexemes[line_ix][satellite_pos] = satellite_lexeme

    # TODO: Look at this more. I believe this is where the original bigram elements get replaced with
    # But both the original and the satellite will get replaced with the bigram,
    # which seems different than the original paper


## Frequency updater

Now we handle the frequency updater, which we use for handling the statistics and log log_likelihood.

```python
self.frequency_updater.set_new_freqs_for_elements_in_winner(
    self.all_lexemes, self.winner_info, self.merge_token_count
)
self.frequency_updater.process_bgrs_with_same_element_types_as_winner(
    self.gapsize, self.all_tables, self.all_bigrams
)
self.frequency_updater.add_new_bigrams(self.new_bigrams)
self.frequency_updater.update_conflicting_bigram_freqs(
    self.conflicting_bigrams
)
self.frequency_updater.remove_winner()
```

In [25]:
corpus_size -= merge_token_count


In [26]:
_lexemes_to_freqs[winner_info.el1]


18314

In [27]:
# subtract total count from each element
# FrequencyUpdater.set_new_freqs_for_elements_in_winner
# NOTE: RUnning this several times will continually subtract
el1_freq = _lexemes_to_freqs[winner_info.el1]
new_el1_freq = el1_freq - merge_token_count
_lexemes_to_freqs[winner_info.el1] = new_el1_freq

el2_freq = _lexemes_to_freqs[winner_info.el2]
new_el2_freq = el2_freq - merge_token_count
_lexemes_to_freqs[winner_info.el2] = new_el2_freq


# FrequencyUpdater.process_bgrs_with_same_element_types_as_winner
# get bigrams with elements of winner
test = set()
for curr_gapsize in range(gapsize + 1):
    el1_left_pos = initial_bigrams.left_lex_to_bigrams[(winner_info.el1, curr_gapsize)]
    el1_right_pos = initial_bigrams.right_lex_to_bigrams[
        (winner_info.el1, curr_gapsize)
    ]
    el2_left_pos = initial_bigrams.left_lex_to_bigrams[(winner_info.el2, curr_gapsize)]
    el2_right_pos = initial_bigrams.right_lex_to_bigrams[
        (winner_info.el2, curr_gapsize)
    ]

    cols = ["el1_freq", "el2_freq"] * 2
    bigram_pos = [
        el1_left_pos,
        el1_right_pos,
        el2_left_pos,
        el2_right_pos,
    ]
    for bigrams, col in zip(bigram_pos, cols):
        for bigram in bigrams:
            value = new_el1_freq if col.startswith("el1") else new_el2_freq
            table.at[bigram, col] = value
    # This can definitely be optimized if we could groupby el1, el2
    # rather than a lookup at each individual row


In [28]:
original_el1_freq, original_el2_freq = (13662, 2471)
assert (new_el1_freq, new_el2_freq) == (original_el1_freq, original_el2_freq)


In [29]:
# FrequencyUpdater.add_new_bigrams
from copy import deepcopy

# Going to make a copy just in case we want to validate against the original
collected_bigrams: BigramData = deepcopy(initial_bigrams)

# Bigrams.add(new_bigrams)
# for all these loops: collected = initial (lookup) + new (in iter)
for index, (bgr, freq) in enumerate(new_bigrams.bigrams_to_freqs.items()):
    collected_bigrams.bigrams_to_freqs[bgr] += freq
    curr_locs = initial_bigrams.bigrams_to_locations[bgr]
    collected_bigrams.bigrams_to_locations[bgr] = curr_locs.union(
        new_bigrams.bigrams_to_locations[bgr]
    )

for (el1, curr_gapsize), bigrams in new_bigrams.left_lex_to_bigrams.items():
    curr_left_lex_to_bigrams = initial_bigrams.left_lex_to_bigrams[(el1, curr_gapsize)]
    collected_bigrams.left_lex_to_bigrams[
        (el1, curr_gapsize)
    ] = curr_left_lex_to_bigrams.union(bigrams)

for (el2, curr_gapsize), bigrams in new_bigrams.right_lex_to_bigrams.items():
    curr_right_lex_to_bigrams = initial_bigrams.right_lex_to_bigrams[
        (el2, curr_gapsize)
    ]
    collected_bigrams.right_lex_to_bigrams[
        (el2, curr_gapsize)
    ] = curr_right_lex_to_bigrams.union(bigrams)

collected_bigrams.type_count = len(collected_bigrams.bigrams_to_freqs)

new_bigram_records = []
for bigram in new_bigrams.bigrams_to_freqs:
    new_freq = collected_bigrams.bigrams_to_freqs[bigram]
    new_el1_freq = _lexemes_to_freqs[bigram.el1]
    new_el2_freq = _lexemes_to_freqs[bigram.el2]
    # FrequencyUpdater.new_data_for_tables.push_row
    # Which just adds all these to one list - I think I can just append

    # FrequencyUpdater.all_tables.add
    # This can be made much simpler since we arent doing the broken tables
    new_bigram_records.append(
        {
            "bgr": bigram,
            "bgr_freq": new_freq,
            "el1_freq": new_el1_freq,
            "el2_freq": new_el2_freq,
        }
    )


new_bigram_df = pd.DataFrame(new_bigram_records).set_index("bgr")
table = pd.concat((table, new_bigram_df), axis="rows")


In [30]:
# FrequencyUpdater.update_conflicting_bigram_freqs
# all_bigrams.deduct_freqs(self.conflicting_bigrams)
# Here all bigrams is collected_bigrams (which has been modified above)

for index, (bgr, freq) in enumerate(conflicting_bigrams.bigrams_to_freqs.items()):
    collected_bigrams.bigrams_to_freqs[bgr] -= freq
    curr_locs = conflicting_bigrams.bigrams_to_locations[bgr]
    for loc in curr_locs:
        collected_bigrams.bigrams_to_locations[bgr].remove(loc)
    if collected_bigrams.bigrams_to_freqs[bgr] < 1:
        del collected_bigrams.bigrams_to_freqs[bgr]
        del collected_bigrams.bigrams_to_locations[bgr]
        collected_bigrams.left_lex_to_bigrams[(bgr.el1, bgr.gapsize)].remove(bgr)
        collected_bigrams.right_lex_to_bigrams[(bgr.el2, bgr.gapsize)].remove(bgr)

collected_bigrams.type_count = len(collected_bigrams.bigrams_to_freqs)

for bigram in conflicting_bigrams.bigrams_to_freqs:
    table.at[bigram, "bgr_freq"] = collected_bigrams.bigrams_to_freqs[bigram]


In [31]:
# FrequencyUpdater.remove_winner()
del collected_bigrams.bigrams_to_freqs[winner_info]
del collected_bigrams.bigrams_to_locations[winner_info]
collected_bigrams.left_lex_to_bigrams[(winner_info.el1, winner_info.gapsize)].remove(
    winner_info
)
collected_bigrams.right_lex_to_bigrams[(winner_info.el2, winner_info.gapsize)].remove(
    winner_info
)

collected_bigrams.type_count = len(collected_bigrams.bigrams_to_freqs)

# 537076
collected_bigrams.type_count


537076

**✨ That's an iteration!**